In [1]:
colab = False

if colab == True:
    !pip install nnunetv2
    !pip install torchio
    

In [2]:
import os
import torch
from torch.utils.data import random_split
import torchio as tio
from pathlib import Path
import importlib
import shutil
from collections import OrderedDict
import json
import numpy as np
import nnunetv2

print('current working directory:', os.getcwd())

if colab:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)
    base_dir = '/content/drive/My Drive/Colab Notebooks/tumor-segmentation'
    os.chdir(base_dir)
    torch.cuda.empty_cache()

else:
    base_dir = '/home/asp/Downloads/DMIAI/DMIAI_2023/tumor-segmentation'

from utils import validate_segmentation, plot_prediction, dice_score
import torchio_utils
importlib.reload(torchio_utils)
from torchio_utils import torchio_compose_train, plot_example

In [3]:
# Set the environment variables
os.environ['nnUNet_raw'] = os.path.join(base_dir, 'nnUNet_raw')
os.environ['nnUNet_preprocessed'] = os.path.join(base_dir, 'nnUNet_preprocessed')
os.environ['nnUNet_results'] = os.path.join(base_dir, 'nnUNet_results')

## THE FOLLOWING ONLY ON COLAB!!!

In [6]:
#!nnUNetv2_plan_and_preprocess -h
!nnUNetv2_plan_and_preprocess -d 2 --verify_dataset_integrity

Fingerprint extraction...
Dataset002
Using <class 'nnunetv2.imageio.natural_image_reager_writer.NaturalImage2DIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.natural_image_reager_writer.NaturalImage2DIO'> as reader/writer
100%|██████████████████████████████████████| 1000/1000 [00:09<00:00, 102.12it/s]
Experiment planning...
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 16, 'patch_size': array([384, 512]), 'median_image_size_in_voxels': array([331., 488.]), 'spacing': array([1., 1.]), 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': (2, 2, 2, 2, 2, 2, 2), 'n_conv_per_stage_decoder': (2, 2, 2, 2, 2, 2), 'num_pool_per_axis': [6, 6], 'pool_op_

In [7]:
!nnUNetv2_train -h

usage: nnUNetv2_train [-h] [-tr TR] [-p P]
                      [-pretrained_weights PRETRAINED_WEIGHTS]
                      [-num_gpus NUM_GPUS] [--use_compressed] [--npz] [--c]
                      [--val] [--val_best] [--disable_checkpointing]
                      [-device DEVICE]
                      dataset_name_or_id configuration fold

positional arguments:
  dataset_name_or_id    Dataset name or ID to train with
  configuration         Configuration that should be trained
  fold                  Fold of the 5-fold cross-validation. Should be an int
                        between 0 and 4.

options:
  -h, --help            show this help message and exit
  -tr TR                [OPTIONAL] Use this flag to specify a custom trainer.
                        Default: nnUNetTrainer
  -p P                  [OPTIONAL] Use this flag to specify a custom plans
                        identifier. Default: nnUNetPlans
  -pretrained_weights PRETRAINED_WEIGHTS
                        [O

## The following only works with CUDA

In [8]:
# !nnUNetv2_train Dataset001 2d 4 -device 'cpu'